# Exploring the Shared Datasets in the LSST Science Platform
<br>Owner(s): **Phil Marshall** ([@drphilmarshall](https://github.com/LSSTScienceCollaborations/StackClub/issues/new?body=@drphilmarshall)), 
<br>Last Verified to Run: **2018-08-05**
<br>Verified Stack Release: **16.0**

In this notebook we'll take a look at some of the datasets available on the LSST Science Platform. 

### Learning Objectives:

After working through this tutorial you should be able to: 
1. Start figuring out which of the available datasets is going to be of most use to you in any given project; 

When it is finished, you should be able to:
2. Plot the patches and tracts in a given dataset on the sky;
3. List the available catalogs in a given dataset.

### Logistics
This notebook is intended to be runnable on `lsst-lspdev.ncsa.illinois.edu` from a local git clone of https://github.com/LSSTScienceCollaborations/StackClub.

## Set-up

We'll need the `stackclub` package to be installed. If you are not developing this package, you can install it using `pip`, like this:
```
pip install git+git://github.com/LSSTScienceCollaborations/StackClub.git#egg=stackclub
```
If you are developing the `stackclub` package (eg by adding modules to it to support the Stack Club tutorial that you are writing, you'll need to make a local, editable installation. In the top level folder of the `StackClub` repo, do:

In [1]:
! cd .. && python setup.py -q develop --user && cd -

/home/pjm43/notebooks/.beavis/StackClub/Basics


When editing the `stackclub` package files, we want the latest version to be imported when we re-run the import command. To enable this, we need the %autoreload magic command.

In [2]:
%load_ext autoreload
%autoreload 2

For accessing the datasets using the Butler, and then visualizing the results, we'll need the following modules:

In [3]:
%matplotlib inline
# %matplotlib ipympl

import os, glob
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from IPython.display import IFrame, display, Markdown

import stackclub

In [4]:
import lsst.daf.persistence as dafPersist
import lsst.daf.base as dafBase

import lsst.afw.math as afwMath
import lsst.afw.geom as afwGeom

import lsst.afw.detection as afwDetect
import lsst.afw.image as afwImage
import lsst.afw.table as afwTable

import lsst.afw.display as afwDisplay

You can find the Stack version that this notebook is running by using eups list -s on the terminal command line:

In [5]:
# What version of the Stack am I using?
! echo $HOSTNAME
! eups list -s | grep lsst_distrib

jld-lab-pjm43-r160
lsst_distrib          16.0+1     	current v16_0 setup


## Listing the Available Datasets
First, let's look at what is currently available. There are two primary shared dataset folders in the LSP, the read-only `/datasets` folder, and the group-writeable folder `/projects/shared/datasets`. Let's see what's in there

**`/projects/shared/data`:** These datasets are designed to be small test sets, ideal for tutorials.

In [6]:
shared_datasets = ! ls -d /project/shared/data/* | grep -v README
shared_datasets

['/project/shared/data/afwdata',
 '/project/shared/data/ci_hsc',
 '/project/shared/data/COSMOS_catalogs',
 '/project/shared/data/DATA_ci_hsc',
 '/project/shared/data/gaia_dr2',
 '/project/shared/data/gaia_dr2_1am',
 '/project/shared/data/HSC_overlaps_db',
 '/project/shared/data/Twinkles_subset',
 '/project/shared/data/with-globular']

In [7]:
%%bash
shared_datasets=$( ls -d /project/shared/data/* | grep -v README )
for dataset in $shared_datasets; do
    du -sh $dataset
done

6.4G	/project/shared/data/afwdata
8.0G	/project/shared/data/ci_hsc
2.9G	/project/shared/data/COSMOS_catalogs
8.0G	/project/shared/data/DATA_ci_hsc
552G	/project/shared/data/gaia_dr2
414G	/project/shared/data/gaia_dr2_1am
105M	/project/shared/data/HSC_overlaps_db
6.4G	/project/shared/data/Twinkles_subset
624M	/project/shared/data/with-globular


**`/datasets`:**
These are typically much bigger: to measure the size, uncomment the second cell below and edit it to target the dataset you are interested in. Running `du` on all folders takes several minutes.

In [8]:
datasets = ! ls -d /datasets/* | grep -v USAGE | grep -v html
datasets

['/datasets/all-sky',
 '/datasets/auxTel',
 '/datasets/comCam',
 '/datasets/ctio0m9',
 '/datasets/decam',
 '/datasets/des_sn',
 '/datasets/gapon',
 '/datasets/goo',
 '/datasets/hsc',
 '/datasets/lsstCam',
 '/datasets/refcats',
 '/datasets/sdss']

In [9]:
# %%bash
# datasets=$( ls -d /datasets/* | grep -v USAGE | grep -v html )
# for dataset in $datasets; do
#     du -h $dataset
# done

## Visualizing Sky Coverage
In this section, we'll plot the available patches and tracts in a given dataset on the sky, following the LSST DESC tutorial [dm_butler_skymap.ipynb](https://github.com/LSSTDESC/DC2-analysis/blob/master/tutorials/dm_butler_skymap.ipynb). In fact, we will _import_ this notebook, so that we can re-use its functions. This operation is handled by the `stackclub.wimport` function.

In [10]:
dm_butler_skymap_notebook = "https://github.com/LSSTDESC/DC2-analysis/raw/master/tutorials/dm_butler_skymap.ipynb"

skymapper = stackclub.wimport(dm_butler_skymap_notebook, vb=True)

Returning path to module: .downloads/dm_butler_skymap.ipynb


> BUG: remote notebooks are not yet `wimport`-able. A workaround could be to import the downloaded file explicitly. This is not yet working, hence the commented out failed attempt below.

In [11]:
# import sys, os
# import stackclub
# sys.path.append(os.getcwd() + '/.downloads')

In [12]:
# import dm_butler_skymap

Now we can attempt to plot the available tracts, using the `plot_skymap_tract()` function.

In [13]:
# repo = "/project/shared/data/Twinkles_subset/output_data_v2"
repo = "/datasets/hsc/repo/rerun/DM-13666/WIDE"
butler = dafPersist.Butler(repo)

# Glob the merged coadd folder for the tracts that have data.  Unfortunately, this information is not
# directly accessible from the data butler.
tracts = sorted([int(os.path.basename(x)) for x in
                 glob.glob(os.path.join(repo, 'deepCoadd-results', 'merged', '*'))])

# How many tracts do we have?
print("Found {} tracts".format(len(tracts)))

/opt/lsst/software/stack/stack/miniconda3-4.3.21-10a4fa6/Linux64/meas_mosaic/16.0+1/python/lsst/meas/mosaic/utils.py:30: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

The backend was *originally* set to 'module://ipykernel.pylab.backend_inline' by the following code:
  File "/opt/lsst/software/stack/python/miniconda3-4.3.21/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/opt/lsst/software/stack/python/miniconda3-4.3.21/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/opt/rh/rh-python36/root/usr/lib/python3.6/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/opt/rh/rh-python36/root/usr/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.st

Found 91 tracts


In [14]:
"""
Uncomment this cell when the `wimport` bug is fixed (or avoided).

# Now, loop over all the tracts, plotting them as gray, numbered, rectangles:
ax = None
for tract in tracts:
    skyMap = butler.get('deepCoadd_skyMap')
    ax = skymapper.plot_skymap_tract(skyMap, tract=tract, title='', ax=ax)
""";

## Summary

In this notebook we took a first look at the datasets available to us in two shared directories in the LSST science platform filesystem.